In [1]:
!pip install -q trulens_eval llama_index llama-index-llms-openai llama_hub llmsherpa llama-cpp-python

In [2]:
# Import the os module to interact with the operating system
import os

# Import the load_dotenv function from the dotenv module
from dotenv import load_dotenv

# Call the load_dotenv function to load environment variables from a .env file
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

In [3]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [4]:
from trulens_eval import Tru
tru = Tru()

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [5]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.113.117:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Start with basic RAG.

In [6]:
import glob

In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [8]:
documents = []
for file in glob.glob("/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data" + "/*.pdf"):
    loader = PyPDFLoader(file)
    pages = loader.load_and_split()
    documents.extend(pages)

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents = SimpleDirectoryReader("data").load_data()

In [10]:
documents

[Document(id_='9eeabcee-56e2-4a2a-9201-7bc75b1e3b87', embedding=None, metadata={'page_label': '1', 'file_name': 'q1_2023.pdf', 'file_path': '/Users/priyanshutuli/Desktop/RAG_pipeline_testing/data/q1_2023.pdf', 'file_type': 'application/pdf', 'file_size': 3438112, 'creation_date': '2024-04-12', 'last_modified_date': '2024-04-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text=' \n   \n  \n \n \n              \n     \xa0  \n              \n     \xa0  \n   \n   \n        \n     \n      \n      \n     \xa0\n        \n       \n  \n      \n         \n                 \n    \n                \n     \n                  \n                  \n                  \n                  \n                  \n              \n                            \n                

In [11]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [12]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer=AutoTokenizer.from_pretrained(model)
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="cpu",
    eos_token_id=tokenizer.eos_token_id
    )



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import Prompt

In [24]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided"""

In [25]:
query_wrapper_prompt = Prompt("<[USER|>{query_str}<|ASSISTANT|]")

In [26]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False, "top_p": 0},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [27]:
from llama_index.core import ServiceContext
from llama_index.core import VectorStoreIndex, StorageContext, Document

# initialize llm
llm = llm

# # knowledge store
document = Document(text="\n\n".join([doc.text for doc in documents]))
print(document)

# service context for index
service_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

# create index
index = VectorStoreIndex.from_documents([document], service_context=service_context)

Doc ID: 55d1268b-ec63-451a-9f57-f31d01d180f8
Text: UNITED STATES  SECURITIES AND EXCHANGE COMMISSION  Washington,
D.C. 20549  FORM 10-Q  (Mark One)  ☑ QUARTERLY REPORT PURSUANT TO
SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  For the
quarterly period ended March 31, 2023  OR  ☐ TRANSITION REPORT
PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transit...


In [28]:
system_prompt = Prompt("We have provided context information below that you may use. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Please answer the question: {query_str}\n")

# basic rag query engine
rag_basic = index.as_query_engine(text_qa_template = system_prompt)

## Load test set

In [29]:
honest_evals = [
    "What are the typical coverage options for homeowners insurance?",
    "What are the requirements for long term care insurance to start?",
    "Can annuity benefits be passed to beneficiaries?",
    "Are credit scores used to set insurance premiums? If so, how?",
    "Who provides flood insurance?",
    "Can you get flood insurance outside high-risk areas?",
    "How much in losses does fraud account for in property & casualty insurance?",
    "Do pay-as-you-drive insurance policies have an impact on greenhouse gas emissions? How much?",
    "What was the most costly earthquake in US history for insurers?",
    "Does it matter who is at fault to be compensated when injured on the job?"
]

## Set up Evaluation

In [30]:
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

In [42]:
embed_model = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [32]:
import numpy as np
from trulens_eval import Tru, Feedback, TruLlama, Langchain

tru = Tru()

# start fresh
tru.reset_database()

from trulens_eval.feedback import Groundedness

In [35]:
from langchain_community.llms import HuggingFaceHub

repo_id = "HuggingFaceH4/zephyr-7b-beta"
READER_MODEL_NAME = "zephyr-7b-beta"

READER_LLM = HuggingFaceHub(
    repo_id=repo_id,
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

/Users/priyanshutuli/Desktop/RAG_pipeline_testing/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(


In [36]:
lc = Langchain(READER_LLM)

In [37]:
from trulens_eval.feedback.provider.hugs import Huggingface
huggingface_provider = Huggingface()

In [40]:
qa_relevance = (
    Feedback(lc.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input_output()
)

qs_relevance = (
    Feedback(lc.relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(TruLlama.select_source_nodes().node.text)
    .aggregate(np.mean)
)

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .


In [48]:
# embedding distance
from trulens_eval.feedback import Embeddings


# embed = Embeddings(embed_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))
# f_embed_dist = (
#     Feedback(embed.cosine_distance)
#     .on_input()
#     .on(TruLlama.select_source_nodes().node.text)
# )

from trulens_eval.feedback import Groundedness

grounded = Groundedness(groundedness_provider=huggingface_provider)

f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name="Groundedness")
        .on(TruLlama.select_source_nodes().node.text.collect())
        .on_output()
        .aggregate(grounded.grounded_statements_aggregator)
)

# honest_feedbacks = [qa_relevance, qs_relevance, f_groundedness]

# from trulens_eval import FeedbackMode

# tru_recorder_rag_basic = TruLlama(
#         rag_basic,
#         app_id='1) Basic RAG - Honest Eval',
#         feedbacks=honest_feedbacks
#     )

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1007)>


In [46]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.113.117:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [49]:
tru.stop_dashboard()

In [47]:
# Run evaluation on 10 sample questions
with tru_recorder_rag_basic as recording:
    for question in honest_evals:
        response = rag_basic.query(question)

NameError: name 'tru_recorder_rag_basic' is not defined

In [ ]:
tru.get_leaderboard(app_ids=["1) Basic RAG - Honest Eval"])

Our simple RAG often struggles with retrieving not enough information from the insurance manual to properly answer the question. The information needed may be just outside the chunk that is identified and retrieved by our app.